
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [5]:
import requests
from dataclasses import dataclass, asdict
from typing import List
import json
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from datetime import datetime, date, timedelta
import dateparser

def convert_date(date_txt):
    dt = None
    if date_txt == "вчера":
        today = date.today()
        dt = today - timedelta(days=1)
    elif date_txt == "сегодня":
        dt = date.today()
    else:
        dt = dateparser.parse(date_txt).date()
    return dt


def get_links(url, start_from_date):
    start_from_date = datetime.strptime(start_from_date, '%Y-%m-%d').date()
    page = 1
    all_links = []
    while True:
        prfx = "" if page == 1 else f"page{page}"
        news = requests.get(f"{url}{prfx}")
        if news.status_code == 404:
            break
        soup = BeautifulSoup(news.text, 'html.parser')
        articals = soup.find_all('article', class_='post post_preview')
        for artical in articals:
            date_text = artical.find('span', class_="post__time").text.split(" в ")[0].strip()
            dt = convert_date(date_text)
            if dt >= start_from_date:
                link = artical.find('a', class_='post__title_link').get("href")
                all_links.append(link)
            else:
                return all_links

        page += 1
        time.sleep(0.3)
    return


KEYWORDS = ['музыка', 'VK']
base_url = "https://habr.com/ru/news/"
all_links = get_links(base_url, "2020-12-03")
kom_news = pd.DataFrame()

for link in all_links[:len(all_links)]:  # ибо долго
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = pd.to_datetime(soup.find(class_='post__time').get('data-time_published'), yearfirst=True).date()
    title = soup.find(class_='post__title-text').text
    text = soup.find(id='post-content-body').text
    if any([re.search(kwd, text) for kwd in KEYWORDS]):
        row = {'date': date, 'title': title, "link": link, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])

print(kom_news)

         date                                              title  \
0  2020-12-06    Власти Беларуси опять пытаются глушить Telegram   
0  2020-12-05  Spotify научила модель на основе ИИ искать пла...   
0  2020-12-04  Постаматы PickPoint стали автоматически открыв...   
0  2020-12-04  Правообладатели обвинили Яндекс.Музыку в невып...   

                                 link  \
0  https://habr.com/ru/news/t/531522/   
0  https://habr.com/ru/news/t/531424/   
0  https://habr.com/ru/news/t/531414/   
0  https://habr.com/ru/news/t/531362/   

                                                text  
0  Сегодня примерно в 11:35 все операторы связи Б...  
0  \n\r\nSpotify подала пантентную заявку на моде...  
0  В работе сервиса PickPoint произошел сбой из-з...  
0  Правообладатели обвинили сервис Яндекс.Музыка ...  


Задание 2.
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [3]:
@dataclass
class MailChecker:
    emailAddresses: List[str]
df = pd.DataFrame()
EMAILS = ['bro@gmail.com', 'nick@gmail.com']
url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
headers = {
"Access-Control-Expose-Headers": "Vaar-Version, Vaar-Status",
"Vaar-Header-App-Product": "hackcheck-web-avast",
"Vaar-Version": "0",
"Vaar-Status": "0",
}
data = asdict(MailChecker(EMAILS))
s = requests.Session()
resp = s.post(url, json=data, headers=headers)

breaches = resp.json()["breaches"]
summary = resp.json()["summary"]

def filtr_email(row, summary):
    for k,v in summary.items():
        if row['breachId'] in v["breaches"]:
            return k

branch_df = pd.DataFrame(breaches).transpose()
branch_df["email"] = branch_df.apply(filtr_email, args=(summary,), axis=1 )
branch_df["date"] = branch_df["description"].apply(lambda x: x.split(",")[0].replace("In ",""))
branch_df = branch_df.reset_index()[["email", "date","site","description"]].sort_values("email")

print(branch_df)

             email                    date                   site  \
0    bro@gmail.com               July 2019        cprewritten.net   
82   bro@gmail.com           December 2012    heroesofnewerth.com   
81   bro@gmail.com            October 2018               pluto.tv   
79   bro@gmail.com                May 2017             edmodo.com   
78   bro@gmail.com                May 2020           homechef.com   
..             ...                     ...                    ...   
58  nick@gmail.com           November 2020  iquiquepropiedades.cl   
90  nick@gmail.com           November 2020      frankkoretsky.com   
38  nick@gmail.com           December 2018          storybird.com   
36  nick@gmail.com           November 2020           iitbrain.com   
25  nick@gmail.com  At an unconfirmed date          trackmill.com   

                                          description  
0   In July 2019, virtual world Club Penguin Rewri...  
82  In December 2012, the online MMOG Heroes of Ne...  
81 